In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import pylab as pl
df = pd.read_csv("/kaggle/input/co2-emission-by-vehicles/CO2 Emissions_Canada.csv")
df.head()

In [ ]:
df.info()

In [ ]:
corr = df.corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr,annot=True)

In [ ]:
Ft = pd.get_dummies(df['Fuel Type'],drop_first=True,prefix='Fuel')
df = df.drop(['Fuel Type'],axis=1)
df = pd.concat([df,Ft],axis=1)
Tr = pd.get_dummies(df['Transmission'],drop_first=True)
df = df.drop(['Transmission'],axis=1)
df = pd.concat([df,Tr],axis=1)

In [ ]:
X = df.drop(['CO2 Emissions(g/km)','Fuel Consumption Comb (mpg)'],axis=1)
y = df['CO2 Emissions(g/km)']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = 0.8)

In [ ]:
cat_cols = ['Make','Model','Vehicle Class']
import category_encoders as ce
target_enc = ce.CatBoostEncoder(cols = cat_cols)
target_enc.fit(x_train[cat_cols],y_train)

In [ ]:
train_enc = target_enc.transform(x_train[cat_cols])
test_enc = target_enc.transform(x_test[cat_cols])

x_train = x_train.drop(['Make','Model','Vehicle Class'],axis=1)
x_test = x_test.drop(['Make','Model','Vehicle Class'],axis=1)
x_train = pd.concat([x_train,train_enc],axis=1)
x_test = pd.concat([x_test,test_enc],axis=1)

In [ ]:
x_train.head()

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler # For Mx=0 and Dx=1
from sklearn.svm import SVC
model = make_pipeline(StandardScaler(), SVC(C = 1000000, random_state = 0)) # create SVM classifier
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print('Accuracy (train): ', model.score(x_train, y_train))
print('Accuracy (test) : ', model.score(x_test, y_test))

In [ ]:
# Let's compare with the case when C = 10 (make the range smaller)
model2 = make_pipeline(StandardScaler(), SVC(C = 10, random_state = 0)) # create SVM classifier
model2.fit(x_train, y_train)
y_pred = model2.predict(x_test)
print('Accuracy (train): ', model2.score(x_train, y_train))
print('Accuracy (test) : ', model2.score(x_test, y_test))

In [ ]:
model4 = make_pipeline(StandardScaler(), SVC(C = 1000000, kernel = 'rbf', random_state = 0)) # create SVM classifier
model4.fit(x_train, y_train)
y_pred = model4.predict(x_test)
print('Accuracy (train): ', model4.score(x_train, y_train))
print('Accuracy (test) : ', model4.score(x_test, y_test))
y_pred

In [ ]:
model4 = make_pipeline(StandardScaler(), SVC(C = 1, kernel = 'linear', random_state = 0)) # create SVM classifier
model4.fit(x_train, y_train)
y_pred = model4.predict(x_test)
print('Accuracy (train): ', model4.score(x_train, y_train))
print('Accuracy (test) : ', model4.score(x_test, y_test))
y_pred